In [2]:
import os
import numpy as np
import gensim

In [3]:
d_name = 'ogbg-molfreesolv'

In [4]:
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname) as f:
        for i, line in enumerate(f):
            tokens = line.split()
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(f'{d_name}.txt'))

In [37]:
len(train_corpus)

642

In [38]:
model = gensim.models.doc2vec.Doc2Vec(
    vector_size=100,
    window=7
    )

In [39]:
model.build_vocab(train_corpus)

In [40]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [41]:
X = [model.dv[node] for node in range(len(model.dv))]

In [42]:
from ogb.graphproppred import GraphPropPredDataset

###

dataset = GraphPropPredDataset(name=d_name)
split_idx = dataset.get_idx_split()

y = [data[1] for data in dataset]

In [43]:
train_idx = split_idx['train']
test_idx = split_idx['test']

train_X = np.array([X[i] for i in range(len(X)) if i in train_idx])
test_X = np.array([X[i] for i in range(len(X)) if i in test_idx])

train_y = np.array([y[i] for i in range(len(y)) if i in train_idx]).ravel()
test_y = np.array([y[i] for i in range(len(y)) if i in test_idx]).ravel()

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn import tree

clf = RandomForestClassifier()
clf.fit(train_X, train_y)

ValueError: Unknown label type: 'continuous'

In [ ]:
y_pred = clf.predict(test_X)

y_pred[:3]

In [ ]:
np.bincount(y_pred)

In [ ]:
np.bincount(test_y)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, clf.predict(test_X))